# Applied Examples

In this final section we will cover some applied examples with NumPy. We will cover a few "from scratch" examples as well as using the scikit-learn library. Hopefully this will sell you on the idea using arrays and vectorized math can be useful and practical. 

## Linear Regression with Hill Climbing 

While this is not the most efficient algorithm available for linear regression, it is a cool exercise nonetheless that teaches concepts applicable to other problems. More specifically, we are going to learn hill climbing. The idea behind hill climbing is to make random adjustments to a solution, and if those adjustments result in an improvement, we keep them. We do this thousands or millions or times until the solution does not improve anymore. 

A simple linear regression solves for coefficients $ \beta_0 $ and $ \beta_1 $ given input data $ x $ and output data $ y $. We try to assess a linear relationship between $ x $ and $ y $ and fit the data accordingly. 

$$
\Large y = \Large \beta_0 + \Large \beta_1 x
$$

Let's import these two columns of data $ x $ and $ y $ off of github and save them to two NumPy arrays. 

In [ ]:
import numpy as np
import pandas as pd

data = pd.read_csv("https://bit.ly/2KF29Bd").values
x = data[:,0]
y = data[:,-1]

In [ ]:
x

In [ ]:
y

Next we are going to declare the beta coefficients `b` as a NumPy array which will initialize both $ \beta_0 $ and $ \beta_1 $ as $ 0.0 $. We will also run our hill climbing for 150,000 iterations (`epochs`) and start our loss at a REALLY high number. This will make more sense shortly. 



In [ ]:
# Building the model
b0 = np.array([0.0, 0.0]) 

epochs = 150000  # The number of iterations to perform
n = float(data.shape[0])  # Number of points
best_loss = 10000000000000.0  # Initialize with a really large value

We are going to make our objective a loss function, more specifically the sum of squares. For a given line with $ \beta_0 $ and $ \beta_1 $, we calculate the differences between the line and the given data points, and then square and sum those differences. This sum of squares is what we want to minimize. For every random adjustment to $ \beta_0 $ and $ \beta_1 $, we will calculate the sum of squares and see if it has been reduced. If not, we undo those random adjustments.

But how do we randomly adjust $ \beta_0 $ and $ \beta_1 $? We can use random values from the normal distribution, with a mean of 0 and standard deviation of 1. This will create a volum of mostly small adjustments near 0, but occasionally we make larger adjustments on the tails in the positive and negative directions. 

In [ ]:
for i in range(epochs):

    # Randomly adjust "m" and "b"
    random_adj = np.random.normal(loc=0,scale=1,size=2)
    b0 += random_adj
    
    # Calculate loss, which is total sum squared error
    new_loss = ((y - (b0[0] + b0[1] * x)) ** 2).sum()
    
    # If loss has improved, keep new values. Otherwise revert.
    if new_loss < best_loss:
        print(f"y = {b0[0]} + {b0[1]}x")
        best_loss = new_loss
    else:
        b0 -= random_adj
        
print(f"y = {b0[0]} + {b0[1]}x")


To quickly see the quality of the fit, let's use matplotlib to see how well the line fit to the data points. 

In [ ]:
# plot 
import matplotlib.pyplot as plt
import numpy as np 
# show in chart

plt.plot(x, y, 'o') # scatterplot
plt.plot(x, b0[1]*x + b0[0]) # line
plt.show()

## Linear Regression with Scikit-Learn

While hill climbing can be used to solve a variety of problems, linear regression has some shortcuts using linear algebra methods which we cover [in another course](https://learning.anaconda.cloud/linear-algebra). But for now, we can use scikit-learn and leverage NumPy arrays to pass the data. 

Below, we read the same CSV from the previous example and extract out the `X` and `y` columns. We then fit a `LinearRegression` and get the coefficients from `coef_` and `intercept_`. However, those will return multidimensional arrays so we flatten them. 

In [ ]:
from sklearn.linear_model import LinearRegression
import pandas as pd 
import matplotlib.pyplot as plt

df = pd.read_csv("https://bit.ly/2KF29Bd", delimiter=",")

# Extract input variables (all rows, all columns but last column)
x = df.values[:, :-1]

# Extract output column (all rows, last column)
y = df.values[:, -1]

# Fit a line to the points
fit = LinearRegression().fit(x, y)

# m = 1.7867224, b = -16.51923513
b1 = fit.coef_.flatten()
b0 = fit.intercept_.flatten()
print(f"b1 = {b1}")
print(f"b0 = {b0}")

# show in chart
plt.plot(x, y, 'o') # scatterplot
plt.plot(x, b1*x+b0) # line
plt.show()

## Customer Queue Simulation

In this example, we are going to simulate customers walking into a bank, grocery store, etc. where they are processed one-a-time. To keep things simple we will only have a single clerk to process each customer, although you are free to adapt this to account for multiple clerks. The purpose of this simulation is to see whether a line will form and get out-of-hand, and we could theoretically use this to predict average wait times of customers. 

Recall we studied many probability distributions. The normal distribution might make sense for the customer processing time, assuming the processing time follows a normal distribution. But what about the flow of customers into the store? The exponential distribution will  model how much time lapses between each customer walking in. 

Let's build the simulation below using the normal distribution and exponential distribution. The customers will take on average 3 minutes to process with a standard deviation of .5 minutes. We will model 20 customers on average arriving every hour, but to be consistent in minutes that is $ 1/3 $ of a customer every minute. We will run the simulation for the first 100 customers. 

Note this is a little involved but the idea is we are using these two distributions to create a "realistic" simulation. Run the simulation and observe its output before you dive into the code itself. 

In [ ]:
import numpy as np
from numpy.random import normal, exponential

np.random.seed(0) # use random seed to run identical "random" simulations

mean_checkout_time = 3  # minutes
std_checkout_time = .5  # minutes
mean_arrival_rate = 20 / 60  # customers per minute
customer_ct = 100

# customer arrival times relative to the previous customer
customer_time_betweens = exponential(scale=1/mean_arrival_rate, size=customer_ct+2) # need to add 2 to prevent out-of-index errors

# customer arrival times as minutes since start of simulation
customer_arrival_times = np.cumsum(customer_time_betweens)

# customer checkout times
customer_checkout_times = normal(loc=mean_checkout_time, scale=std_checkout_time, size=customer_ct+2) # need to add 2 to prevent out-of-index errors

# start time at 0 but jump to first customer arrival, and track whether customer is being processed
# and which customers are waiting
current_time = customer_arrival_times[0]
waiting_customers = []

arrived_customer_i = 0
processing_customer_i = 0
processing_customer_start_time = customer_arrival_times[0]

# process customers but stop when all customers have arrived 
while arrived_customer_i < customer_ct:

    # arrival time of processing customer
    processing_cust_arr_tm = customer_arrival_times[processing_customer_i]

    # scheduled finish time of processing customer
    processing_cust_fin_tm = processing_customer_start_time + \
                             customer_checkout_times[processing_customer_i]

    # time of next customer arrival
    def next_cust_arr_tm(): return customer_arrival_times[arrived_customer_i+1]

    # CHECK WHICH EVENT OCCURRED BY MATCHING THE TIMES
    next_event_time = None

    # if the first customer
    if current_time == processing_customer_start_time:
        print(f"{current_time}: CUSTOMER {arrived_customer_i} ARRIVED, NO LINE, PROCESSING IMMEDIATELY")
        next_event_time = np.min([processing_cust_fin_tm, next_cust_arr_tm()])

    # if a customer arrives
    elif current_time == next_cust_arr_tm():
        arrived_customer_i +=1 # increment the arrived customer index

        # if there is no queue and the arriving customer is next
        if processing_customer_i == arrived_customer_i:
            processing_customer_start_time = current_time
            processing_cust_fin_tm = processing_customer_start_time + customer_checkout_times[processing_customer_i]

            print(f"{current_time}: CUSTOMER {arrived_customer_i} ARRIVED, NO LINE, PROCESSING IMMEDIATELY")
        # else there is a queue and the customer must wait in line
        else:
            waiting_customers.append(arrived_customer_i)
            print(f"{current_time}: CUSTOMER {arrived_customer_i} ARRIVED, ADDING TO LINE {waiting_customers}")

        # schedule next event time to be the processing customer finishing or the next customer arrival
        next_event_time = np.min([processing_cust_fin_tm, next_cust_arr_tm()])

    # if a customer finishes processing
    elif current_time == processing_cust_fin_tm:

        # if queue is not empty, take customer out of queue
        if waiting_customers:
            waiting_customers.pop(0)
            print(f"{current_time}: CUSTOMER {processing_customer_i} FINISHED, CUSTOMER {processing_customer_i + 1}"
                  f" REMOVED FROM LINE {waiting_customers}")

            processing_customer_start_time = current_time

            # next event is this customer finishing or the next customer arrival
            next_event_time = np.min([processing_customer_start_time + customer_checkout_times[processing_customer_i +1],
                                      next_cust_arr_tm()])

        else:
            # if the queue is empty, wait for next customer 
            print(f"{current_time}: CUSTOMER {processing_customer_i} FINISHED, WAITING FOR CUSTOMER {processing_customer_i+1}")
            next_event_time = next_cust_arr_tm()

        processing_customer_i += 1 # process next customer

    # move forward to next event
    current_time = next_event_time

When you run the simulation above, you will see a line does irrecoverably build up after enough time has passed. This should tell you another teller might be necessary to process customers! You can also experiment with shorter processing times or longer times in-between customers, and you will find there is an ideal balance at some point where the processing keeps up with the queue.

## Neural Network with scikit-learn 

Let's look at one final example of using NumPy against scikit-learn. Let's first bring in some imports. 

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix

Let's bring in the MNIST dataset. Notice each row is an image of a handwritten digit, although this is not quite obvious yet. But we do see each column holds a pixel value for each image/row. The only exception is the last column, which is the label of what digit that image represents between 0 through 9. 

In [ ]:

df = pd.read_csv('https://bit.ly/3ilJc2C', compression='zip', delimiter=",")
df

These pixel values represent how darkened it is, on a scale between 0 and 255. Let's divide by 255 so it is between 0 and 1. Let's also grab the input columns of pixels as `x`, and the output column of the labels `y`.  

In [ ]:

x = df.values[:, :-1] / 255.0
y = df.values[:, -1]

In machine learning, it is good practice to set aside part of the data as the test dataset (e.g., 1/3 of the data), and use the remaining data as the training dataset (e.g., 2/3). This way we can later use the test data to evaluate how well the model works on data it has not seen before. The `train_test_split()` will do this for us and return four NumPy arrays serving as the train/test datasets. 

In [ ]:
# Separate training and testing data
# each class is proportionally represented in both sets
X_train, X_test, Y_train, Y_test = train_test_split(x, y,
    test_size=1/3, random_state=10)

Finally, we will create a classification neural network `MLPClassifier` from scikit-learn. We will pass the `X_train` and `Y_train` data to the model, use 100 hidden nodes in a single hidden layer, and use a logistic activation function for the hidden layer. If you are not familiar with machine learning or neural networks, [there is an Anaconda class teaching that topic](https://learning.anaconda.cloud/getting-started-with-ai-ml). 

Let's train the model, and then evaluate the accuracy the test data. We can go a step further and create a confusion matrix, which tracks how often each digit was correctly identified, and when they were not what digits they were misclassified as. The confusion matrix itself will be returned as a NumPy array. 

In [ ]:

nn = MLPClassifier(solver='sgd',
                   hidden_layer_sizes=(100, ),
                   activation='logistic',
                   max_iter=480,
                   learning_rate_init=.1)

nn.fit(X_train, Y_train)

print(f"Test set score: {nn.score(X_test, Y_test)}")

cf = confusion_matrix(y_true=Y_test, y_pred=nn.predict(X_test))
print(cf)

That should give us plenty of examples to ponder. Hopefully you see that NumPy is a building block to perform many tasks and work with many libraries like scikit-learn. 

## Exercise

Complete the code below to perform a linear regression on [this dataset](https://raw.githubusercontent.com/thomasnield/machine-learning-demo-data/master/regression/linear_normal.csv) on Github. The left column is the `x` input variable, and the right column is the `y` output variable. 

Complete by replacing the question marks `?` below, including extracting the coefficients. 

In [ ]:
from sklearn.linear_model import LinearRegression
import pandas as pd 
import matplotlib.pyplot as plt

url = r"https://raw.githubusercontent.com/thomasnield/machine-learning-demo-data/master/regression/linear_normal.csv"
df = pd.read_csv(url, delimiter=",")

# Extract input variables (all rows, all columns but last column)
x = ? 

# Extract output column (all rows, last column)
y = ?

# Fit a line to the points
fit = LinearRegression().fit(?, ?)

# m = 1.7867224, b = -16.51923513
b1 = fit.coef_.?
b0 = fit.intercept_.?
print(f"b1 = {b1}")
print(f"b0 = {b0}")

# show in chart
plt.plot(x, y, 'o') # scatterplot
plt.plot(x, b1*x+b0) # line
plt.show()

### SCROLL DOWN FOR ANSWER
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
v 

In [ ]:
from sklearn.linear_model import LinearRegression
import pandas as pd 
import matplotlib.pyplot as plt

url = r"https://raw.githubusercontent.com/thomasnield/machine-learning-demo-data/master/regression/linear_normal.csv"
df = pd.read_csv(url, delimiter=",")

# Extract input variables (all rows, all columns but last column)
x = df.values[:, :-1]

# Extract output column (all rows, last column)
y = df.values[:, -1]

# Fit a line to the points
fit = LinearRegression().fit(x, y)

# m = 1.7867224, b = -16.51923513
b1 = fit.coef_.flatten()
b0 = fit.intercept_.flatten()
print(f"b1 = {b1}")
print(f"b0 = {b0}")

# show in chart
plt.plot(x, y, 'o') # scatterplot
plt.plot(x, b1*x+b0) # line
plt.show()